# Chapter 38: FastAPI Server & MCP Integration

Run this notebook directly in Google Colab - no local Python needed!

**Full code**: [GitHub](https://github.com/eduardd76/AI_for_networking_and_security_engineers/tree/main/CODE/Volume-3-Production-Systems/Chapter-38-FastAPI-Server-MCP)

## Setup

Install dependencies and configure API keys.

In [ ]:
# Install dependencies
!pip install -q fastapi uvicorn pydantic anthropic requests netmiko psutil

# Import and configure API key
import os
from getpass import getpass

# Check for Colab secrets first
try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
    print('✓ Using API keys from Colab secrets')
except:
    # Fall back to manual entry
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Enter ANTHROPIC_API_KEY: ')
    print('✓ API keys configured')

print('\n✅ Setup complete! Ready to run examples.')

## Example 1: Basic FastAPI Server

Create a production FastAPI server with health checks and request validation.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Any
from datetime import datetime
import ipaddress

# Initialize FastAPI app
app = FastAPI(
    title="Network AI Operations API",
    description="Production API for AI-powered network operations",
    version="1.0.0"
)

# Request/Response models
class DeviceQuery(BaseModel):
    """Request model for device queries"""
    device_ip: str = Field(..., description="Device IP address")
    commands: List[str] = Field(..., description="Commands to execute")
    timeout: Optional[int] = Field(30, description="Timeout in seconds")

    @validator('device_ip')
    def validate_ip(cls, v):
        """Validate IP address format"""
        try:
            ipaddress.ip_address(v)
            return v
        except ValueError:
            raise ValueError(f"Invalid IP address: {v}")

    @validator('commands')
    def validate_commands(cls, v):
        """Ensure commands list is not empty"""
        if not v:
            raise ValueError("Commands list cannot be empty")
        return v

class HealthResponse(BaseModel):
    """Health check response"""
    status: str
    timestamp: datetime
    version: str
    uptime_seconds: float

# Global state
START_TIME = datetime.now()

@app.get("/")
async def root():
    """Root endpoint"""
    return {
        "service": "Network AI Operations API",
        "version": "1.0.0",
        "docs": "/docs"
    }

@app.get("/health", response_model=HealthResponse)
async def health_check():
    """Health check endpoint"""
    uptime = (datetime.now() - START_TIME).total_seconds()
    return HealthResponse(
        status="healthy",
        timestamp=datetime.now(),
        version="1.0.0",
        uptime_seconds=uptime
    )

@app.post("/query-device")
async def query_device(query: DeviceQuery):
    """Query network device (simulated)"""
    return {
        "device_ip": query.device_ip,
        "commands_executed": len(query.commands),
        "status": "success",
        "timestamp": datetime.now()
    }

# Test the endpoints
print("FastAPI server configuration:")
print(f"  Title: {app.title}")
print(f"  Version: {app.version}")
print(f"  Endpoints: {len(app.routes)}")
print("\nExample request:")

# Simulate a request
test_query = DeviceQuery(
    device_ip="192.168.1.1",
    commands=["show version", "show ip interface brief"]
)
print(f"  Device IP: {test_query.device_ip}")
print(f"  Commands: {test_query.commands}")
print(f"  Timeout: {test_query.timeout}s")

## Example 2: MCP Server Implementation

Implement Model Context Protocol for standardized tool calling.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any, Optional
from datetime import datetime
import time

app = FastAPI(title="MCP Network Operations Server")

# MCP Protocol Models
class MCPTool(BaseModel):
    """MCP tool definition"""
    name: str
    description: str
    input_schema: Dict[str, Any]

class MCPToolCall(BaseModel):
    """MCP tool invocation request"""
    tool: str
    arguments: Dict[str, Any]

class MCPToolResult(BaseModel):
    """MCP tool invocation result"""
    tool: str
    result: Any
    error: Optional[str] = None
    execution_time_ms: float

# Define available tools
NETWORK_TOOLS = {
    "get_device_config": {
        "name": "get_device_config",
        "description": "Retrieve running configuration from a network device",
        "input_schema": {
            "type": "object",
            "properties": {
                "device_ip": {"type": "string", "description": "IP address of the device"},
                "config_type": {"type": "string", "enum": ["running", "startup"]}
            },
            "required": ["device_ip"]
        }
    },
    "check_interface_status": {
        "name": "check_interface_status",
        "description": "Check status of network interfaces on a device",
        "input_schema": {
            "type": "object",
            "properties": {
                "device_ip": {"type": "string", "description": "IP address of the device"},
                "interface_name": {"type": "string", "description": "Specific interface (optional)"}
            },
            "required": ["device_ip"]
        }
    }
}

async def execute_tool(tool_name: str, arguments: Dict[str, Any]) -> Any:
    """Execute a tool with given arguments (simulated)"""
    if tool_name == "get_device_config":
        device_ip = arguments["device_ip"]
        config_type = arguments.get("config_type", "running")
        return {
            "device": device_ip,
            "config_type": config_type,
            "config": f"!\n! Configuration for {device_ip}\n!\nversion 15.2\nhostname Router1\n...",
            "lines": 247,
            "retrieved_at": datetime.now().isoformat()
        }
    elif tool_name == "check_interface_status":
        device_ip = arguments["device_ip"]
        interface_name = arguments.get("interface_name")
        if interface_name:
            return {
                "device": device_ip,
                "interface": interface_name,
                "status": "up",
                "protocol": "up",
                "ip_address": "192.168.1.1"
            }
        else:
            return {
                "device": device_ip,
                "interfaces": [
                    {"name": "GigabitEthernet0/0", "status": "up", "protocol": "up"},
                    {"name": "GigabitEthernet0/1", "status": "down", "protocol": "down"}
                ]
            }
    else:
        raise ValueError(f"Unknown tool: {tool_name}")

@app.get("/mcp/tools", response_model=List[MCPTool])
async def list_tools():
    """List all available tools"""
    return [
        MCPTool(
            name=tool_data["name"],
            description=tool_data["description"],
            input_schema=tool_data["input_schema"]
        )
        for tool_data in NETWORK_TOOLS.values()
    ]

@app.post("/mcp/call-tool", response_model=MCPToolResult)
async def call_tool(request: MCPToolCall):
    """Execute a tool"""
    start_time = time.time()
    
    if request.tool not in NETWORK_TOOLS:
        return MCPToolResult(
            tool=request.tool,
            result=None,
            error=f"Tool not found: {request.tool}",
            execution_time_ms=0
        )
    
    try:
        result = await execute_tool(request.tool, request.arguments)
        execution_time = (time.time() - start_time) * 1000
        
        return MCPToolResult(
            tool=request.tool,
            result=result,
            error=None,
            execution_time_ms=execution_time
        )
    except Exception as e:
        execution_time = (time.time() - start_time) * 1000
        return MCPToolResult(
            tool=request.tool,
            result=None,
            error=str(e),
            execution_time_ms=execution_time
        )

# Test MCP server
print("MCP Tools Available:")
tools = await list_tools()
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

print("\nTesting tool call:")
test_call = MCPToolCall(
    tool="check_interface_status",
    arguments={"device_ip": "192.168.1.1"}
)
result = await call_tool(test_call)
print(f"  Tool: {result.tool}")
print(f"  Execution time: {result.execution_time_ms:.2f}ms")
print(f"  Result: {result.result}")

## Example 3: AI Agent with MCP Integration

Connect Claude to MCP server for autonomous network operations.

In [ ]:
from anthropic import Anthropic
import json

# Initialize Anthropic client
client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

# Define tools for Claude
CLAUDE_TOOLS = [
    {
        "name": "check_interface_status",
        "description": "Check the operational status of network interfaces. Shows interface state, protocol status, and IP configuration.",
        "input_schema": {
            "type": "object",
            "properties": {
                "device_ip": {"type": "string", "description": "IP address of the device"},
                "interface_name": {"type": "string", "description": "Specific interface (optional)"}
            },
            "required": ["device_ip"]
        }
    },
    {
        "name": "get_device_config",
        "description": "Retrieve running or startup configuration from a network device.",
        "input_schema": {
            "type": "object",
            "properties": {
                "device_ip": {"type": "string", "description": "IP address of the device"},
                "config_type": {"type": "string", "enum": ["running", "startup"]}
            },
            "required": ["device_ip"]
        }
    }
]

async def execute_mcp_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Execute tool via MCP server (simulated)"""
    # In production, this would make HTTP request to MCP server
    return await execute_tool(tool_name, arguments)

async def agent_query(prompt: str, device_ip: str, max_iterations: int = 5) -> Dict[str, Any]:
    """Execute AI agent query with tool calling"""
    messages = [
        {
            "role": "user",
            "content": f"Device IP: {device_ip}\n\nQuery: {prompt}"
        }
    ]
    
    tool_calls = []
    iterations = 0
    
    while iterations < max_iterations:
        iterations += 1
        
        # Call Claude with tools
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=2048,
            tools=CLAUDE_TOOLS,
            messages=messages
        )
        
        # Check if Claude wants to use tools
        if response.stop_reason == "tool_use":
            assistant_content = []
            
            for content_block in response.content:
                if content_block.type == "tool_use":
                    tool_name = content_block.name
                    tool_input = content_block.input
                    tool_use_id = content_block.id
                    
                    # Execute the tool
                    tool_result = await execute_mcp_tool(tool_name, tool_input)
                    
                    tool_calls.append({
                        "tool": tool_name,
                        "input": tool_input,
                        "result": tool_result
                    })
                    
                    assistant_content.append(content_block)
                    
                    # Add tool result to conversation
                    messages.append({"role": "assistant", "content": assistant_content})
                    messages.append({
                        "role": "user",
                        "content": [{
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": json.dumps(tool_result)
                        }]
                    })
                else:
                    assistant_content.append(content_block)
        else:
            # Claude provided final answer
            final_response = ""
            for content_block in response.content:
                if hasattr(content_block, "text"):
                    final_response += content_block.text
            
            return {
                "prompt": prompt,
                "response": final_response,
                "tool_calls": tool_calls,
                "iterations": iterations
            }
    
    return {
        "prompt": prompt,
        "response": "Maximum iterations reached",
        "tool_calls": tool_calls,
        "iterations": iterations
    }

# Test AI agent
print("Testing AI Agent with MCP Tools\n")
result = await agent_query(
    prompt="Check the interface status on this device",
    device_ip="192.168.1.1"
)

print(f"Prompt: {result['prompt']}")
print(f"\nResponse: {result['response']}")
print(f"\nTool Calls: {len(result['tool_calls'])}")
for i, call in enumerate(result['tool_calls'], 1):
    print(f"  {i}. {call['tool']}: {call['input']}")
print(f"\nIterations: {result['iterations']}")

## Example 4: Production Server with Monitoring

Complete production server with health checks and metrics.

In [ ]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from datetime import datetime
import psutil
import time

app = FastAPI(
    title="Production Network AI API",
    description="Production-ready API with monitoring",
    version="1.0.0"
)

# Middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global state
START_TIME = datetime.now()
REQUEST_COUNT = 0

@app.middleware("http")
async def log_requests(request: Request, call_next):
    """Log all requests with timing"""
    global REQUEST_COUNT
    REQUEST_COUNT += 1
    
    start_time = time.time()
    response = await call_next(request)
    duration = (time.time() - start_time) * 1000
    
    print(f"Request {REQUEST_COUNT}: {request.method} {request.url.path} - {duration:.2f}ms")
    return response

@app.get("/health")
async def health_check():
    """Basic health check"""
    uptime = (datetime.now() - START_TIME).total_seconds()
    return {
        "status": "healthy",
        "uptime_seconds": uptime,
        "version": "1.0.0",
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health/detailed")
async def detailed_health():
    """Detailed health check with system metrics"""
    uptime = (datetime.now() - START_TIME).total_seconds()
    
    return {
        "status": "healthy",
        "uptime_seconds": uptime,
        "version": "1.0.0",
        "timestamp": datetime.now().isoformat(),
        "metrics": {
            "cpu_percent": psutil.cpu_percent(interval=0.1),
            "memory_percent": psutil.virtual_memory().percent,
            "requests_handled": REQUEST_COUNT
        }
    }

@app.get("/metrics")
async def metrics():
    """Prometheus-style metrics"""
    uptime = (datetime.now() - START_TIME).total_seconds()
    
    metrics_text = f"""# HELP service_uptime_seconds Service uptime
# TYPE service_uptime_seconds gauge
service_uptime_seconds {uptime}

# HELP http_requests_total Total HTTP requests
# TYPE http_requests_total counter
http_requests_total {REQUEST_COUNT}

# HELP system_cpu_percent CPU usage percentage
# TYPE system_cpu_percent gauge
system_cpu_percent {psutil.cpu_percent(interval=0.1)}

# HELP system_memory_percent Memory usage percentage
# TYPE system_memory_percent gauge
system_memory_percent {psutil.virtual_memory().percent}
"""
    return metrics_text

# Test the production server
print("Production Server Status:")
health = await health_check()
print(f"  Status: {health['status']}")
print(f"  Uptime: {health['uptime_seconds']:.1f}s")

print("\nDetailed Health Check:")
detailed = await detailed_health()
print(f"  CPU: {detailed['metrics']['cpu_percent']:.1f}%")
print(f"  Memory: {detailed['metrics']['memory_percent']:.1f}%")
print(f"  Requests: {detailed['metrics']['requests_handled']}")

print("\n✅ Production server configured successfully!")

## Next Steps

- Full code: [Chapter 38 on GitHub](https://github.com/eduardd76/AI_for_networking_and_security_engineers/tree/main/CODE/Volume-3-Production-Systems/Chapter-38-FastAPI-Server-MCP)
- Learn more: [vExpertAI.com](https://vexpertai.com)
- Author: Eduard Dulharu ([@eduardd76](https://github.com/eduardd76))

**Production Deployment:**
- Deploy with Docker: `docker build -t network-ai-api .`
- Use Nginx for load balancing
- Enable HTTPS with SSL certificates
- Monitor with Prometheus + Grafana